In [22]:
import pandas as pd
df = pd.read_csv('data/activityRawData.csv')
df.head(4)

,id,nameActivity,rateActivity,numComment,priceActivity,urlImage,city,time,distance,typeActivity
0,0,Nhà thờ Đức Bà Sài Gòn,4.4,22000.0,Miễn phí,https://lh5.googleusercontent.com/p/AF1QipPfkH...,Hồ Chí Minh,NaN,"('10.7797746', '106.616617')",Nhà thờ chính tòa Công giáo
1,1,Dinh Độc Lập,4.4,34000.0,40000,https://lh5.googleusercontent.com/p/AF1QipMSw2...,Hồ Chí Minh,Thứ Sáu 08:00–15:30,"(0, 0)",Bảo tàng
2,2,Khu di tích lịch sử Địa đạo Củ Chi,4.5,12000.0,NaN,https://lh5.googleusercontent.com/p/AF1QipNL_j...,Hồ Chí Minh,Thứ Sáu 07:00–17:00,"('11.1415798', '106.3791944')",Điểm mốc lịch sử
3,3,Nhà hát Thành phố Hồ Chí Minh,4.5,13000.0,NaN,https://lh5.googleusercontent.com/p/AF1QipO1ie...,Hồ Chí Minh,NaN,"('10.7766243', '106.6207635')",Nhà hát Opera


In [23]:
#### Modify priceActivity: have price or set to 0

df['priceActivity'].replace('Miễn phí', 0 , inplace=True)
df['priceActivity'].fillna(0, inplace=True)
df['priceActivity'].isnull().sum()

0

In [24]:
#### Change Time Open - Close 

for i, val in enumerate(df['time']):
    val = str(val)
    if 'Mở cửa cả ngày' in val:
        getOpenTime  = '07:00'
        getCloseTime = '12:00'
    elif 'Thứ Sáu' in val:
        getTime = val.strip('Thứ Sáu').strip()    
        
        if ',' in getTime:
            getOpenTime, getCloseTime = getTime.split(',')
            getOpenTime, _  = getOpenTime.split('–')
            _ ,getCloseTime = getCloseTime.split('–')
        elif '–' in getTime:
            getOpenTime, getCloseTime = getTime.split('–')
        else:
            getOpenTime   = '07:00'
            getCloseTime  = '17:00'

    else:
        getOpenTime   = '07:00'
        getCloseTime  = '17:00'
   
    df.loc[i, 'openTime'] = getOpenTime
    df.loc[i, 'closeTime'] = getCloseTime

df.drop( columns= ['time' ], inplace=True )
display(df['openTime'].unique())
display(df['closeTime'].unique())

array(['07:00', '08:00', '07:30', '09:30', '08:30', '10:00', '05:00',
       '09:00', '06:00', '16:00', '06:30', '07:15', '04:00', '03:00',
       '04:30', '05:30', '18:00', '05:15', '17:30', '15:00', '00:00',
       '13:00', '14:00', '02:00', '17:00', '07:45', '06:45', '08:15',
       '18:30'], dtype=object)

array(['17:00', '15:30', '17:30', '21:30', '23:00', '18:00', '22:00',
       '21:00', '12:00', '19:30', '19:00', '00:00', '16:30', '05:00',
       '17:15', '18:30', '20:30', '16:00', '20:00', '06:00', '17:45',
       '11:00', '11:30', '08:00', '18:15', '22:30', '19:15', '22:15',
       '23:30', '21:15', '02:00', '05:30', '09:00', '01:00'], dtype=object)

##### Standardized Type Activity

In [25]:
''''
Du lịch biển đảo
Du lịch sinh thái
Du lịch tâm linh
Du lịch giải trí
Địa danh lịch sử
Vãn cảnh & Check-in
Du lịch văn hóa & nghệ thuật

'''
type1 = [ 'Biển' , 'biển' , 'đảo' , 'Đảo' , 'Bãi' , 'bãi' ]
type2 = ['thác' , 'Thác' , 'rừng' , 'sinh thái' , 'Sinh thái' , 'hồ' , 'Rừng' , 'núi' , 'Núi' , 'Hồ' , 'thiên nhiên' , 'Cave' ,  'Vườn' , 'Vịnh' , 'Đầm' , 'gió' , 'Gió']
type3 = [ 'chùa' , 'Chùa' , 'Nhà thờ'  , 'Nhà thờ'  , 'Đền' , 'Đền thờ' , 'Chúa' ]
type4 = [ 'công viên' , 'Công viên'  , 'giải trí' , 'Sở thú']
type5 = [ 'lịch sử', 'quân đội' , 'Lịch sử' ,  'chiến tranh']
type6 = [ 'Cầu' , 'Con đường nổi tiếng' , 'Sông' , 'Khu cắm trại' , 'Văn phòng công ty' , 'Bưu điện' , 'Thắng cảnh' ]
type7 = [ 'Nhà hát' , 'nghệ thuật' , 'chợ' , 'Chợ' , 'văn hóa' , 'Tháp' , 'Bảo tàng' , 'Nơi thờ cúng' , 'tượng' , 'Tượng' , 'Phố'   ]


In [26]:
for i, val in enumerate(df['typeActivity']):
    if val == 'Điểm thu hút khách du lịch':
        df.loc[ i , 'keyword'] =  '####'
        
for i, val in enumerate(df['nameActivity']):
    for type in type1:
        if type in val:
            df.loc[ i, 'keyword'] = 'Du lịch biển đảo'
            break
    for type in type2:
        if type in val:
            df.loc[ i, 'keyword'] = 'Du lịch sinh thái'
            break
    for type in type3:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch tâm linh'
            break
    for type in type4:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch giải trí'
            break

    for type in type5:
        if type in val:
            df.loc[ i , 'keyword'] =  'Địa danh lịch sử'
            break
    
    for type in type6:
        if type in val:
            df.loc[ i , 'keyword'] =  'Vãn cảnh & Check-in'
            break
    for type in type7:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch văn hóa & nghệ thuật'
            break

            
for i, val in enumerate(df['typeActivity']):
    for type in type1:
        if type in val:
            df.loc[ i, 'keyword'] = 'Du lịch biển đảo'
            break
    for type in type2:
        if type in val:
            df.loc[ i, 'keyword'] = 'Du lịch sinh thái'
            break
    for type in type3:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch tâm linh'
            break
    for type in type4:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch giải trí'
            break
    for type in type5:
        if type in val:
            df.loc[ i , 'keyword'] =  'Địa danh lịch sử'
            break
    for type in type6:
        if type in val:
            df.loc[ i , 'keyword'] =  'Vãn cảnh & Check-in'
            break
    for type in type7:
        if type in val:
            df.loc[ i , 'keyword'] =  'Du lịch văn hóa & nghệ thuật'
            break

              
df['keyword'].replace( '####' , 'Du lịch sinh thái' , inplace=True)
df.dropna(subset=["keyword"], inplace=True)
df.drop( columns= ['typeActivity' ], inplace=True )
df.rename(columns = {'keyword':'typeActivity'}, inplace = True)
df.head(4)

,id,nameActivity,rateActivity,numComment,priceActivity,urlImage,city,distance,openTime,closeTime,typeActivity
0,0,Nhà thờ Đức Bà Sài Gòn,4.4,22000.0,0,https://lh5.googleusercontent.com/p/AF1QipPfkH...,Hồ Chí Minh,"('10.7797746', '106.616617')",07:00,17:00,Du lịch tâm linh
1,1,Dinh Độc Lập,4.4,34000.0,40000,https://lh5.googleusercontent.com/p/AF1QipMSw2...,Hồ Chí Minh,"(0, 0)",08:00,15:30,Du lịch văn hóa & nghệ thuật
2,2,Khu di tích lịch sử Địa đạo Củ Chi,4.5,12000.0,0,https://lh5.googleusercontent.com/p/AF1QipNL_j...,Hồ Chí Minh,"('11.1415798', '106.3791944')",07:00,17:00,Địa danh lịch sử
3,3,Nhà hát Thành phố Hồ Chí Minh,4.5,13000.0,0,https://lh5.googleusercontent.com/p/AF1QipO1ie...,Hồ Chí Minh,"('10.7766243', '106.6207635')",07:00,17:00,Du lịch văn hóa & nghệ thuật


In [27]:
df.columns

Index(['id', 'nameActivity', 'rateActivity', 'numComment', 'priceActivity',
       'urlImage', 'city', 'distance', 'openTime', 'closeTime',
       'typeActivity'],
      dtype='object')

In [28]:
df.rename(columns = {'id':'idActivity'}, inplace = True)


In [29]:
len(df)

1404

In [30]:
df.to_csv('data/activity.csv', index=False)